Maciej Łodziński  
Laboratiorium nr.2 - WSI

# Wstęp:
- Celem eksperymentu jest implementacja algorytmu genetycznego z selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną,   
  zbadanie działania owego algorytmu na przykładzie zadania lądowania rakietą oraz wpływu parametru prawdopodobieństwa krzyżowania na  
  ostateczny wynik.  

- Funkcja celu, która określa jakość wyznaczonego rozwiązania zależała od zużycia paliwa podczas lądowania, a także czy rakieta bezpiecznie 
  wylądowała, rozbiła się czy może po upłynięciu czasu nadal jest w powietrzu. Wszystkim trzem przypadkom funkcja przypisuje odpowiednie wartości.  

- Algorytm genetyczny w każdej iteracji wyznacza nową populację rozwiązań, ocenia je i wybiera najlepsze rozwiązanie, jakie pojawiło się  
  podczas działania algorytmu.  

In [ ]:
import numpy as np
from math import ceil
from random import choices

In [2]:
class GeneticAlgorithmSolver:
    def __init__(self, t_max, pc, pm, u):
        self.t_max = t_max
        self.pc = pc
        self.pm = pm
        self.u = u
        
    def get_parameters(self):
        return {"t_max":self.t_max, "pc":self.pc, "pm":self.pm, "u":self.u}
    
    def solve(self, problem, pop0):
        t = 0
        P = pop0
        O = score(problem, P)
        x_best, o_best = find_best(P, O)
        while t < self.t_max:
            S = selection(P, O)
            M = cross_and_mutate(S, self.pc, self.pm, self.u)
            O = score(q, M)
            x, o = find_best(M, O)
            #print_best(t, self.t_max, o)
            if o > o_best:
                x_best = x
                o_best = o
            P = M
            t+=1
        return x_best
    

In [3]:
def q(x):
    v = 0
    gravity = -0.09
    height = 200
    mass = 200 + sum(x)

    for i in range(len(x)):
        mass -= x[i]
        acceleration = 45/mass * x[i]
        v += (acceleration + gravity)
        height += v
        if height < 0:
            return -1000 - sum(x)
        elif height < 2 and abs(v) < 2:
            return 2000 - sum(x)
        
    #Could also return the same value as if crashed because it's the case where the rocket can't land safely and has no fuel left
    return -sum(x)


In [4]:
def score(q, P):
    return [q(x) for x in P]

def create_population(n, u):
    return [np.random.randint(0, 2, n).tolist() for _ in range(u)]

def print_best(t, t_max, o):
    if t%ceil(t_max / 10) == 0:
        print(f'Iteration {t} -> {o}')

def find_best(P, O):
    o_best = max(O)
    index = O.index(o_best)
    x_best = P[index]
    return x_best, o_best

def crossover(p1, p2, pc):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < pc:
        pt = np.random.randint(1, len(p1)-2)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

def mutation(x, pm):
    for i in range(len(x)):
        if np.random.rand() < pm:
            x[i] = 1 - x[i]
    return x

def selection(P, O):
    O = [(o - min(O))/(max(O)-min(O)) for o in O]
    fitness = sum(O)
    probabilities = [o/fitness for o in O]
    probabilities = np.array(probabilities)
    return choices(P, weights=probabilities, k=len(P))

def cross_and_mutate(S, pc, pm, u):
    children = list()
    for i in range(0, u, 2):
        p1, p2 = S[i], S[i+1]
        for c in crossover(p1, p2, pc):
            c = mutation(c, pm)
            children.append(c)
    return children


In [5]:
def test(t_max, pc, pm, u, test_num):
    ans = []
    for i in range(test_num):
        solver = GeneticAlgorithmSolver(t_max, pc, pm, u)
        pop0 = P = create_population(200, u)
        x = solver.solve(q, pop0)
        res = q(x)
        ans.append(res)
        if (i%2==0):
            print(f'Iteration {i} result-> {res}')
    print(f'\nAverage = {sum(ans)/test_num}')
    print(f'Std = {np.round(np.std(ans))}')
    print(f'Min = {np.min(ans)}')
    print(f'Max = {np.max(ans)}')
    

# Badanie wpływu prawdopodobieństwa krzyżowania "pc" na wynik

**Ze względu na to, że algorytmy genetyczne wykorzystują losowość, nie wolno wyciągać wniosków na    
podstawie wyników pojedynczego uruchomienia. Należy porównywać średnie - w tym przypadku z 25 uruchomień.    
Aby jeszcze lepiej ukazać wpływ parametru na wynik, poza średnią, wyliczam także odchylenie standardowe,  
oraz najlepszy i najgorszy ze znalezionych wyników.**


**Test dla pc = 0.0**

In [6]:
test(30, 0.0, 0.3, 20, 25)

Iteration 0 result-> 1915
Iteration 2 result-> 1917
Iteration 4 result-> 1917
Iteration 6 result-> 1916
Iteration 8 result-> 1920
Iteration 10 result-> 1918
Iteration 12 result-> 1920
Iteration 14 result-> 1914
Iteration 16 result-> 1916
Iteration 18 result-> 1916
Iteration 20 result-> 1914
Iteration 22 result-> 1916
Iteration 24 result-> 1918

Average = 1917.08
Std = 2.0
Min = 1914
Max = 1921


**Test dla pc = 0.25**

In [7]:
test(30, 0.25, 0.3, 20, 25)

Iteration 0 result-> 1915
Iteration 2 result-> 1917
Iteration 4 result-> 1917
Iteration 6 result-> 1916
Iteration 8 result-> 1920
Iteration 10 result-> 1917
Iteration 12 result-> 1913
Iteration 14 result-> 1916
Iteration 16 result-> 1914
Iteration 18 result-> 1918
Iteration 20 result-> 1919
Iteration 22 result-> 1915
Iteration 24 result-> 1917

Average = 1916.64
Std = 2.0
Min = 1913
Max = 1920


**Test dla pc = 0.5**

In [8]:
test(30, 0.5, 0.3, 20, 25)

Iteration 0 result-> 1916
Iteration 2 result-> 1914
Iteration 4 result-> 1920
Iteration 6 result-> 1915
Iteration 8 result-> 1916
Iteration 10 result-> 1915
Iteration 12 result-> 1918
Iteration 14 result-> 1920
Iteration 16 result-> 1915
Iteration 18 result-> 1917
Iteration 20 result-> 1916
Iteration 22 result-> 1914
Iteration 24 result-> 1921

Average = 1917.04
Std = 3.0
Min = 1914
Max = 1925


**Test dla pc = 0.75**

In [9]:
test(30, 0.75, 0.3, 20, 25)

Iteration 0 result-> 1916
Iteration 2 result-> 1919
Iteration 4 result-> 1922
Iteration 6 result-> 1916
Iteration 8 result-> 1917
Iteration 10 result-> 1916
Iteration 12 result-> 1915
Iteration 14 result-> 1916
Iteration 16 result-> 1915
Iteration 18 result-> 1922
Iteration 20 result-> 1920
Iteration 22 result-> 1918
Iteration 24 result-> 1916

Average = 1917.44
Std = 3.0
Min = 1913
Max = 1923


**Test dla pc = 1.0**

In [10]:
test(30, 1.0, 0.3, 20, 25)

Iteration 0 result-> 1919
Iteration 2 result-> 1920
Iteration 4 result-> 1917
Iteration 6 result-> 1919
Iteration 8 result-> 1917
Iteration 10 result-> 1917
Iteration 12 result-> 1916
Iteration 14 result-> 1918
Iteration 16 result-> 1922
Iteration 18 result-> 1916
Iteration 20 result-> 1917
Iteration 22 result-> 1916
Iteration 24 result-> 1916

Average = 1917.0
Std = 2.0
Min = 1914
Max = 1922


# Wpływ parametru "pc" na wyniki
**Wraz ze wzrostem wartości prawdopodobieństwa krzyżowania w całym jego zakresie rośnie średnia wartość najlepszego rozwiązania,
jednakże optymalna wartość tego parametru zależy od konkretnego problemu. W tym przypadku jest to jednak na tyle mała różnica, że równie dobrze
może być to kwestia losowości w algorytmie, specyficznej funkcji kosztu, która bardzo często zwraca prawie najlepszy wynik lub faktu, że przeprowadzamy krzyżowanie tylko jednopunktowe.**

# Wnioski:
- **widoczny wpływ losowości na podstawie odchylenia standardowego przez co algorytm powinien być uruchamiany wielokrotnie**
- **stosunkowo szybki algorytm**
- **wiele parametrów przez co można mieć problem z ich dobraniem do konkretnego problemu**